In [2]:
#for loading data
import pandas as pd

#for llm
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#logging
import pickle

#similarity
from sentence_transformers import SentenceTransformer

#visualization
#plot matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load data

In [54]:
job_statements = pd.read_excel("datasets/task_statements.xlsx")
job_statements.columns = job_statements.columns.str.lower()
job_statements = job_statements.drop(labels=["incumbents responding","date","domain source"], axis=1).rename(columns={"o*net-soc code":"code", "task type":"type", "task id": "id"})


In [59]:
# set up occupation data
occupations = pd.read_excel('datasets/occupation_data.xlsx').dropna()
occupations.columns = occupations.columns.str.lower()
#rename the column 
occupations = occupations.rename(columns={'o*net-soc code':'code'})   
#drop rows with "all other" in the content
occupations = occupations[~occupations['title'].str.contains("All Other")]
#change data type
occupations['code'] = occupations['code'].astype(str)
occupations['title'] = occupations['title'].astype(str)
occupations['description'] = occupations['description'].astype(str)

test_occ = occupations.sample(1, random_state=1).iloc[0].title



In [61]:
def get_des (title):
    task_list = list(job_statements.query("title == @title")["task"])
    return task_list

In [62]:
get_des(test_occ)

['Confer with customers by telephone or in person to provide information about products or services, take or enter orders, cancel accounts, or obtain details of complaints.',
 'Keep records of customer interactions or transactions, recording details of inquiries, complaints, or comments, as well as actions taken.',
 "Check to ensure that appropriate changes were made to resolve customers' problems.",
 'Contact customers to respond to inquiries or to notify them of claim investigation results or any planned adjustments.',
 'Determine charges for services requested, collect deposits or payments, or arrange for billing.',
 'Complete contract forms, prepare change of address records, or issue service discontinuance orders, using computers.',
 'Refer unresolved customer grievances to designated departments for further investigation.',
 "Resolve customers' service or billing complaints by performing activities such as exchanging merchandise, refunding money, or adjusting bills.",
 'Review in